In [1]:
import numpy as np
import importlib
import util as uti
import matplotlib.pyplot as plt
importlib.reload(uti)

# import math lib
from math import pi

# import Qiskit
from qiskit import Aer, IBMQ, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# import basic plot tools
#import qiskit.visualization as vis
from qiskit.visualization import plot_histogram
%matplotlib inline

/home/kinga/.local/lib/python3.6/site-packages/qiskit/__init__.py:67: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of the 0.17.0 release. Support for running Qiskit with Python 3.6 will be removed in a future release.
  "future release.", DeprecationWarning)


# input data

In [9]:
train1 = np.array([0.8, 0.2])
train2 = np.array([0.1, 0.9])
test = np.array([0.2, 0.8])

# quantum k-means with swap test

In [10]:
def calc_amplitudes(v):
    norm = np.linalg.norm(v)
    return v/norm

## swap test

In [11]:
# routine that expresses overlap of v1 and v2
def swap_test(v1_amp, v2_amp, draw=False):
    
    backend = Aer.get_backend('qasm_simulator')
    # create Quantum Register called "qr" with 5 qubits
    qr = QuantumRegister(5, name="qr")
    # create Classical Register called "cr" with 5 bits
    cr = ClassicalRegister(1, name="cr")
    # Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
    # and your Classical Register "cr"
    qc = QuantumCircuit(qr, cr, name="swap_test")
    
    # load data
    qc.initialize(v1_amp, 1)
    # second qubit stays in |0> because second vector has only 1 qubit and the other is padding
    qc.initialize(v2_amp, qr[3:5])
    #qc.initialize(v2_amp[2:3], 4)
    
    # build swap test
    qc.h(qr[0])
    qc.cswap(qr[0], qr[1], qr[3])
    qc.cswap(qr[0], qr[2], qr[4])
    qc.h(qr[0])
    
    # measure
    qc.measure(qr[0], cr[0])
    if draw:
        qc.draw()
        
    shots=1024    
    job = execute(qc, backend=backend, shots=shots)
#   print( result.get_data(qc))
    counts = job.result().get_counts()
    print(counts)
    return ((counts['0']/shots) - 0.5)*2

In [16]:
def quantum_state_init(v1, v2):
    v1_norm = np.linalg.norm(v1)
    v2_norm = np.linalg.norm(v2)
    Z = v1_norm**2 + v2_norm**2
    q1_state = np.array([v1_norm, v2_norm]) * (1/np.sqrt(Z))
    q2_state = np.hstack([calc_amplitudes(v1), calc_amplitudes(v2)]) * (1/np.sqrt(2))
    return (q1_state, q2_state, Z)

In [17]:
def calc_dist(overlap, Z):
    return 2*Z*np.abs(overlap)**2

In [1]:
# create oracle that flips amplitude of qubit corresponding to cluster number cluster_winner
def create_cluster_oracle(cluster_winner):
    qr = QuantumRegister(1, name="qr")
    qc = QuantumCircuit(qr, name="cluster_oracle")
    # if cluster idx == 0, make matrix [-1 0; 0 1], which corresponds to gate sequence XZX
    if cluster_winner == 0:
        qc.x(qr)
        qc.z(qr)
        qc.x(qr)
        return qc
    # if cluster idx == 1, make matrix [1 0; 0 -1], which corresponds to gate Z
    elif cluster_winner == 1:
        qc.z(qr)
        return qc

In [2]:
# create combined oracle for grover search
def create_combined_oracle(distances):
    # would actually be distances[0] * create_cluster_oracle(0) + distances[1] * create_cluster_oracle(1)
    # to pick right oracle, but here looking directly at it
    if distances[0]:
        return create_cluster_oracle(0)
    else:
        return create_cluster_oracle(1)

In [ ]:
# only need to pass in oracle, no inputs, since algo is applied to all possible inputs at once in any case
def grover_search(oracle, qubits_n=2):
    grover_circuit = QuantumCircuit(qubits_n)
    # initialize all qubits to uniform amplitudes by applying Hadamard gate
    for q in range(qubits_n):
        qc.h(q)
    

## calculate this in a loop k times to get k distances to the k clusters

In [20]:
# load/initialize two training vectors on qubits 2-3 and 4-5
q1_state, q2_state, Z = quantum_state_init(train1, train2)
print(q1_state)
print(q2_state)
overlap = swap_test(q1_state, q2_state, draw=True)

[0.67330033 0.7393691 ]
[0.68599434 0.17149859 0.07808688 0.70278193]
{'1': 351, '0': 673}


In [21]:
overlap

0.314453125

In [22]:
distance = calc_dist(overlap, Z)
distance

0.296642303466797

In [7]:
# build dummy circuit for drawing
# create Quantum Register called "qr" with 5 qubits
qr = QuantumRegister(5, name="qr")
# create Classical Register called "cr" with 5 bits
cr = ClassicalRegister(1, name="cr")
# Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
# and your Classical Register "cr"
qc = QuantumCircuit(qr, cr, name="swap_test")

# load data
qc.initialize(q1_state, 1)
# second qubit stays in |0> because second vector has only 1 qubit and the other is padding
qc.initialize(q2_state, qr[3:5])
#qc.initialize(v2_amp[2:3], 4)
qc.barrier()

# build swap test
qc.h(qr[0])
qc.cswap(qr[0], qr[1], qr[3])
qc.cswap(qr[0], qr[2], qr[4])
qc.h(qr[0])

# measure
qc.measure(qr[0], cr[0])
qc.draw()


░ ┌───┐      ┌───┐┌─┐
qr_0: ─────────────────────────────────────────────────░─┤ H ├─■──■─┤ H ├┤M├
               ┌────────────────────────────┐          ░ └───┘ │  │ └───┘└╥┘
qr_1: ─────────┤ initialize(0.6733,0.73937) ├──────────░───────X──┼───────╫─
               └────────────────────────────┘          ░       │  │       ║ 
qr_2: ─────────────────────────────────────────────────░───────┼──X───────╫─
      ┌──────────────────────────────────────────────┐ ░       │  │       ║ 
qr_3: ┤0                                             ├─░───────X──┼───────╫─
      │  initialize(0.68599,0.1715,0.078087,0.70278) │ ░          │       ║ 
qr_4: ┤1                                             ├─░──────────X───────╫─
      └──────────────────────────────────────────────┘ ░                  ║ 
cr: 1/════════════════════════════════════════════════════════════════════╩═
                                                                          0